In [ ]:
from pathlib import Path

import numpy as np
from ase.io import read

In [ ]:
import sys

sys.tracebacklimit = None

# Parameters & constants

In [ ]:
# Control parameters
debug = False
use_mpi = False


# Initial structure
structure_label = "BLG"
input_path = Path("examples/graphene/rectangular")
initial_structure = read(input_path / "POSCAR", format="vasp")

# Distances
distances = [0.0, 1.425, 2.468, 2.850]

# Supercell parameters

workdir = Path("output/graphene/rectangular")

## lengths
lx = 50  # length in x direction (Å)
ly = lx / np.sqrt(3)  # length in y direction (Å) keeping the b/a ratio
lz = 10  # length in z direction (Å)
# workdir /= f"len_{lx}x{int(ly)}"

## or, repetitions
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction
workdir /= f"rep_{nx}x{ny}"

# Hamiltonian parameters
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlayer_coupling = 0.33
nearest_neighbor = 1  # degree of nearest neighbors | don't use 0!

workdir /= f"nn_{nearest_neighbor}"

workdir.mkdir(parents=True, exist_ok=True)

# Define structure

In [ ]:
from tb_hamiltonian.utils import get_structure

structure = get_structure(
    initial_structure=initial_structure,
    # lengths=(lx, ly, lz),
    repetitions=(nx, ny, nz),
)

# This will show up at top of Hamiltonian output file
structure.info["label"] = structure_label

structure.write(workdir.parent / "POSCAR", format="vasp")

# Compute H

In [ ]:
from tb_hamiltonian import TBHamiltonian

H = TBHamiltonian(
    structure=structure,
    nearest_neighbor=nearest_neighbor,
    distances=distances,
    hopping_parameters=hopping_parameters,
    interlayer_coupling=interlayer_coupling,
    debug=debug,
)

H.build()

# Apply onsite term

In [ ]:
from tb_hamiltonian.potentials import PotentialFactory

potential = PotentialFactory("kronig-penney")
potential.params = {
    "amplitude": 1.0,
    # "width": 0.5,
}

H.update_onsite_terms(
    onsite_term=0.0,
    potential=potential,
    alpha=(1.0,),
)

workdir = (
    workdir
    / f"{potential.name}"
    / f"amplitude_{potential.params['amplitude']}"
    # / f"width_{potential.params['width']}"
)

workdir.mkdir(parents=True, exist_ok=True)

# Sanity checks


In [ ]:
# H.onsite_count()

In [ ]:
# H.interaction_counts()

# Write H to file

In [ ]:
H.write_to_file(workdir, use_mpi=use_mpi)

# Compute band structure

In [ ]:
band_structure = H.get_band_structure(
    high_sym_points={
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    path="Γ P X W Y Γ W",
    points_per_segment=10,
    use_sparse_solver=False,
    sparse_solver_params={"k": 4, "sigma": 1e-8},
    use_mpi=use_mpi,
)

# Plotting

In [ ]:
# H.plot_matrix()

In [ ]:
# H.plot_grid()

In [ ]:
# H.plot_potential()

In [ ]:
# H.plot_bands(band_structure)